# Memory-Efficient ML Pipeline for Farm To Feed Dataset (Redo)

This notebook implements a memory-efficient machine learning pipeline to predict customer purchasing behavior for 1-week and 2-week windows using pandas, gc, and LightGBM. Includes FileUpload widgets for easy file handling in Colab.

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
from sklearn.metrics import roc_auc_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from ipywidgets import FileUpload
import io

In [ ]:
# File Upload Widgets
upload_train = FileUpload(accept='.csv', multiple=False, description='Train.csv')
upload_test = FileUpload(accept='.csv', multiple=False, description='Test.csv')
upload_customer = FileUpload(accept='.csv', multiple=False, description='customer_data.csv')
upload_sku = FileUpload(accept='.csv', multiple=False, description='sku_data.csv')

display(upload_train, upload_test, upload_customer, upload_sku)

FileUpload(value={}, accept='.csv', description='Train.csv')

FileUpload(value={}, accept='.csv', description='Test.csv')

FileUpload(value={}, accept='.csv', description='customer_data.csv')

FileUpload(value={}, accept='.csv', description='sku_data.csv')

In [ ]:
# Step 1: Efficient Data Loading & Downcasting
print("Loading data...")
if upload_train.value:
    train = pd.read_csv(io.BytesIO(list(upload_train.value.values())[0]['content']))
else:
    train = pd.read_csv('Train.csv')

if upload_test.value:
    test = pd.read_csv(io.BytesIO(list(upload_test.value.values())[0]['content']))
else:
    test = pd.read_csv('Test.csv')

if upload_customer.value:
    customer = pd.read_csv(io.BytesIO(list(upload_customer.value.values())[0]['content']))
else:
    customer = pd.read_csv('customer_data.csv')

if upload_sku.value:
    sku = pd.read_csv(io.BytesIO(list(upload_sku.value.values())[0]['content']))
else:
    sku = pd.read_csv('sku_data.csv')

def downcast(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    for col in df.select_dtypes(include=['int64']).columns:
        max_val = df[col].max()
        if max_val < 2**8:
            df[col] = df[col].astype('int8')
        elif max_val < 2**16:
            df[col] = df[col].astype('int16')
        else:
            df[col] = df[col].astype('int32')
    return df

train = downcast(train)
test = downcast(test)
customer = downcast(customer)
sku = downcast(sku)

train['week_start'] = pd.to_datetime(train['week_start'])
test['week_start'] = pd.to_datetime(test['week_start'])
customer['customer_created_at'] = pd.to_datetime(customer['customer_created_at'])

gc.collect()

Loading data...


NameError: name 'upload_train' is not defined

In [ ]:
# Merge additional data
train = train.merge(customer, on='customer_id', how='left', suffixes=('', '_cust'))
train = train.merge(sku, on='product_unit_variant_id', how='left', suffixes=('', '_sku'))
test = test.merge(customer, on='customer_id', how='left', suffixes=('', '_cust'))
test = test.merge(sku, on='product_unit_variant_id', how='left', suffixes=('', '_sku'))

gc.collect()

In [ ]:
# Step 2: Smart Grid Creation & Feature Engineering
full_df = pd.concat([train[['customer_id', 'product_unit_variant_id', 'week_start', 'qty_this_week']],
                     test[['customer_id', 'product_unit_variant_id', 'week_start']].assign(qty_this_week=0.0)])
full_df = downcast(full_df)
full_df = full_df.sort_values(['customer_id', 'product_unit_variant_id', 'week_start'])

print("Creating features...")
full_df['lag1_qty'] = full_df.groupby(['customer_id', 'product_unit_variant_id'])['qty_this_week'].shift(1)
full_df['lag2_qty'] = full_df.groupby(['customer_id', 'product_unit_variant_id'])['qty_this_week'].shift(2)
full_df['lag3_qty'] = full_df.groupby(['customer_id', 'product_unit_variant_id'])['qty_this_week'].shift(3)
full_df['rolling_mean_4'] = full_df.groupby(['customer_id', 'product_unit_variant_id'])['qty_this_week'].rolling(4).mean().reset_index(0, drop=True)
full_df['rolling_max_4'] = full_df.groupby(['customer_id', 'product_unit_variant_id'])['qty_this_week'].rolling(4).max().reset_index(0, drop=True)

full_df[['lag1_qty', 'lag2_qty', 'lag3_qty', 'rolling_mean_4', 'rolling_max_4']] = full_df[['lag1_qty', 'lag2_qty', 'lag3_qty', 'rolling_mean_4', 'rolling_max_4']].fillna(0)

gc.collect()

In [ ]:
# Merge features back
feature_cols = ['lag1_qty', 'lag2_qty', 'lag3_qty', 'rolling_mean_4', 'rolling_max_4']
train = train.merge(full_df[['customer_id', 'product_unit_variant_id', 'week_start'] + feature_cols],
                    on=['customer_id', 'product_unit_variant_id', 'week_start'], how='left')
test = test.merge(full_df[['customer_id', 'product_unit_variant_id', 'week_start'] + feature_cols],
                  on=['customer_id', 'product_unit_variant_id', 'week_start'], how='left')

del full_df
gc.collect()

In [ ]:
# Step 3: Target Generation
print("Creating targets...")
train = train.sort_values(['customer_id', 'product_unit_variant_id', 'week_start'])
train['target_purchase_1w'] = (train.groupby(['customer_id', 'product_unit_variant_id'])['qty_this_week'].shift(-1) > 0).astype(int)
train['target_qty_1w'] = train.groupby(['customer_id', 'product_unit_variant_id'])['qty_this_week'].shift(-1).fillna(0)
train['target_purchase_2w'] = (train.groupby(['customer_id', 'product_unit_variant_id'])['qty_this_week'].shift(-2) > 0).astype(int)
train['target_qty_2w'] = train.groupby(['customer_id', 'product_unit_variant_id'])['qty_this_week'].shift(-2).fillna(0)

train = train.dropna(subset=['target_purchase_1w'])

gc.collect()

In [ ]:
# Encode categoricals
cat_cols = ['customer_category', 'customer_status', 'grade_name', 'unit_name']
le = LabelEncoder()
for col in cat_cols:
    combined = pd.concat([train[col], test[col]])
    le.fit(combined)
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

features = ['selling_price', 'customer_category', 'customer_status', 'grade_name', 'unit_name'] + feature_cols

In [ ]:
# Step 4: Modeling
print("Training models...")
weeks = sorted(train['week_start'].unique())
val_weeks = weeks[-2:]
val_mask = train['week_start'].isin(val_weeks)

models = {}
targets = ['target_purchase_1w', 'target_qty_1w', 'target_purchase_2w', 'target_qty_2w']
for target in targets:
    print(f"Training {target}...")
    X = train[features]
    y = train[target]
    X_train, X_val = X[~val_mask], X[val_mask]
    y_train, y_val = y[~val_mask], y[val_mask]

    if 'purchase' in target:
        model = lgb.LGBMClassifier(n_estimators=1000, early_stopping_rounds=50, verbose=-1)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='auc')
        pred_val = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, pred_val)
        print(f"AUC for {target}: {auc}")
    else:
        model = lgb.LGBMRegressor(n_estimators=1000, early_stopping_rounds=50, verbose=-1)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='mae')
        pred_val = model.predict(X_val)
        mae = mean_absolute_error(y_val, pred_val)
        print(f"MAE for {target}: {mae}")

    models[target] = model
    gc.collect()

In [ ]:
# Step 5: Submission
print("Generating predictions...")
test['Target_purchase_next_1w'] = models['target_purchase_1w'].predict_proba(test[features])[:, 1]
test['Target_qty_next_1w'] = models['target_qty_1w'].predict(test[features])
test['Target_purchase_next_2w'] = models['target_purchase_2w'].predict_proba(test[features])[:, 1]
test['Target_qty_next_2w'] = models['target_qty_2w'].predict(test[features])

submission = test[['ID', 'Target_purchase_next_1w', 'Target_qty_next_1w', 'Target_purchase_next_2w', 'Target_qty_next_2w']]
submission.to_csv('submission.csv', index=False)
print("Submission saved.")

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
from sklearn.preprocessing import LabelEncoder

# --- 1. Load Data ---
print("Loading...")
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
customer = pd.read_csv('customer_data.csv') # Load customer data
sku = pd.read_csv('sku_data.csv')

# --- 2. Downcast & Date Conversion ---
def downcast(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    for col in df.select_dtypes(include=['int64']).columns:
        # Adjusted int downcasting to avoid overflow for larger values
        max_val = df[col].max()
        min_val = df[col].min()
        if min_val >= np.iinfo('int8').min and max_val <= np.iinfo('int8').max:
            df[col] = df[col].astype('int8')
        elif min_val >= np.iinfo('int16').min and max_val <= np.iinfo('int16').max:
            df[col] = df[col].astype('int16')
        elif min_val >= np.iinfo('int32').min and max_val <= np.iinfo('int32').max:
            df[col] = df[col].astype('int32')
    return df

train = downcast(train)
test = downcast(test)
customer = downcast(customer) # Downcast customer data
sku = downcast(sku)

train['week_start'] = pd.to_datetime(train['week_start'])
test['week_start'] = pd.to_datetime(test['week_start'])
customer['customer_created_at'] = pd.to_datetime(customer['customer_created_at'])

# --- Merge additional data (customer and sku) into train and test FIRST ---
# This ensures all categorical and customer_created_at columns are present in train/test
train = train.merge(customer, on='customer_id', how='left')
train = train.merge(sku, on='product_unit_variant_id', how='left')
test = test.merge(customer, on='customer_id', how='left')
test = test.merge(sku, on='product_unit_variant_id', how='left')

print("Train columns after merges:", train.columns.tolist())
print("Test columns after merges:", test.columns.tolist())

gc.collect()

# --- 3. Feature Engineering (Unified) ---
# Combine Train and Test to generate Lag Features smoothly
# Drop 'ID' column before concat as it's not a feature and could be duplicated.
test_with_qty = test.assign(qty_this_week=0.0).drop(columns=['ID'], errors='ignore')
full_df = pd.concat([train.drop(columns=['ID'], errors='ignore'), test_with_qty], ignore_index=True)

# The merges with customer and sku are now done earlier on train/test, no need to merge into full_df again here.

full_df = downcast(full_df) # Downcast full_df after concat
full_df = full_df.sort_values(['customer_id', 'product_unit_variant_id', 'week_start'])

print("Full_df columns after concat and sort:", full_df.columns.tolist())

# Generate Lags
print("Generating features...")
grp = full_df.groupby(['customer_id', 'product_unit_variant_id'])['qty_this_week']
full_df['lag1'] = grp.shift(1)
full_df['lag2'] = grp.shift(2)
full_df['roll_mean_4'] = grp.rolling(4).mean().reset_index(level=[0,1], drop=True)

# Fill NaNs
full_df[['lag1', 'lag2', 'roll_mean_4']] = full_df[['lag1', 'lag2', 'roll_mean_4']].fillna(0)

gc.collect()

# --- 4. Create "Stacked" Training Data (The Grandmaster Move) ---
# We transform the data:
# Row 1: Target=Week1, Horizon=1
# Row 2: Target=Week2, Horizon=2

# Ensure all relevant columns are carried over from full_df
# Use all columns that will be features or used for stacking
relevant_cols = ['customer_id', 'product_unit_variant_id', 'week_start', 'qty_this_week',
                 'customer_category_x', 'customer_status_x', 'grade_name_x', 'unit_name_x',
                 'lag1', 'lag2', 'roll_mean_4', 'customer_created_at_x'] # Use _x suffixes for these columns

print("Full_df columns before train_df/test_df split:", full_df.columns.tolist())

train_df = full_df[full_df['week_start'].isin(train['week_start'])][relevant_cols].copy()
test_df = full_df[full_df['week_start'].isin(test['week_start'])][relevant_cols].copy()

# Create Horizon 1 Subset
h1 = train_df.copy()
h1['target_qty'] = h1.groupby(['customer_id', 'product_unit_variant_id'])['qty_this_week'].shift(-1)
h1['target_buy'] = (h1['target_qty'] > 0).astype(int)
h1['horizon'] = 1

# Create Horizon 2 Subset
h2 = train_df.copy()
h2['target_qty'] = h2.groupby(['customer_id', 'product_unit_variant_id'])['qty_this_week'].shift(-2)
h2['target_buy'] = (h2['target_qty'] > 0).astype(int)
h2['horizon'] = 2

# Stack them!
train_stacked = pd.concat([h1, h2], ignore_index=True)
train_stacked = train_stacked.dropna(subset=['target_qty']) # Remove rows where target is NaN (end of data)

# --- 5. Train Unified Models ---
# Add the new merged features to the list of features, including customer_category and customer_status
features = ['lag1', 'lag2', 'roll_mean_4', 'horizon',
            'customer_category_x', 'customer_status_x', 'grade_name_x', 'unit_name_x'] # Use _x suffixes

# Before training, we need to LabelEncode the categorical features
cat_cols_to_encode = ['customer_category_x', 'customer_status_x', 'grade_name_x', 'unit_name_x'] # Use _x suffixes
le = LabelEncoder()
for col in cat_cols_to_encode:
    # Fit on combined data from train_stacked, test_h1, test_h2 for consistent encoding
    # Note: test_df is used for encoding consistency across train_stacked and final test predictions
    combined_data = pd.concat([train_stacked[col], test_df[col]]).astype(str).fillna('UNKNOWN')
    le.fit(combined_data)

    train_stacked[col] = le.transform(train_stacked[col].astype(str).fillna('UNKNOWN'))
    test_df[col] = le.transform(test_df[col].astype(str).fillna('UNKNOWN'))

# Prepare Test Set for H1 and H2 BEFORE encoding customer data to ensure test_df is used correctly
test_h1 = test_df.copy()
test_h2 = test_df.copy()

# Apply encoding to test_h1 and test_h2 based on the encoded test_df (already done above, but confirm this line is still needed for clarity if test_h1/h2 were not direct copies)
# This line ensures test_h1 and test_h2 get the encoded values from test_df
test_h1[cat_cols_to_encode] = test_df[cat_cols_to_encode]
test_h2[cat_cols_to_encode] = test_df[cat_cols_to_encode]

print("Training Unified Classifier...")
clf = lgb.LGBMClassifier(n_estimators=1500, learning_rate=0.05)
clf.fit(train_stacked[features], train_stacked['target_buy'])

print("Training Unified Regressor (Tweedie)...")
reg = lgb.LGBMRegressor(objective='tweedie', n_estimators=1500, learning_rate=0.05)
reg.fit(train_stacked[features], train_stacked['target_qty'])

# --- 6. Predict ---
test_h1['horizon'] = 1
test_h2['horizon'] = 2

submission = test[['ID']].copy()
submission['Target_purchase_next_1w'] = clf.predict_proba(test_h1[features])[:, 1]
submission['Target_qty_next_1w'] = reg.predict(test_h1[features])
submission['Target_purchase_next_2w'] = clf.predict_proba(test_h2[features])[:, 1]
submission['Target_qty_next_2w'] = reg.predict(test_h2[features])

submission.to_csv('submission_horizon.csv', index=False)
print("Done!")

Loading...
Train columns after merges: ['ID', 'customer_id', 'product_unit_variant_id', 'week_start', 'qty_this_week', 'num_orders_week', 'spend_this_week', 'purchased_this_week', 'product_id', 'grade_name_x', 'unit_name_x', 'product_grade_variant_id', 'selling_price', 'customer_category_x', 'customer_status_x', 'customer_created_at_x', 'Target_qty_next_1w', 'Target_purchase_next_1w', 'Target_qty_next_2w', 'Target_purchase_next_2w', 'customer_category_y', 'customer_status_y', 'customer_created_at_y', 'product_name', 'product_grade_variant_sku', 'unit_name_y', 'grade_name_y', 'grade_active_status']
Test columns after merges: ['ID', 'customer_id', 'product_unit_variant_id', 'week_start', 'product_id', 'grade_name_x', 'unit_name_x', 'product_grade_variant_id', 'customer_category_x', 'customer_status_x', 'customer_created_at_x', 'customer_category_y', 'customer_status_y', 'customer_created_at_y', 'product_name', 'product_grade_variant_sku', 'unit_name_y', 'grade_name_y', 'grade_active_stat

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
from sklearn.preprocessing import LabelEncoder

# --- 1. Load Data ---
print("Loading data...")
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
customer = pd.read_csv('customer_data.csv')
sku = pd.read_csv('sku_data.csv')

# --- 2. Downcast & Pre-processing ---
def downcast(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    for col in df.select_dtypes(include=['int64']).columns:
        if df[col].max() < 2**32:
            df[col] = df[col].astype('int32')
    return df

train = downcast(train)
test = downcast(test)
customer = downcast(customer)
sku = downcast(sku)

# Convert Dates
train['week_start'] = pd.to_datetime(train['week_start'])
test['week_start'] = pd.to_datetime(test['week_start'])
customer['customer_created_at'] = pd.to_datetime(customer['customer_created_at'])

# --- 3. Unified Feature Engineering (The "Full Grid" Approach) ---
print("Building Full History...")

# A. Combine Train and Test FIRST
# We need the Test set to exist so that 'shift(-1)' on the last week of Train
# can actually see the first week of Test (if applicable) or handle boundaries correctly.
full_df = pd.concat([
    train[['customer_id', 'product_unit_variant_id', 'week_start', 'qty_this_week']],
    test[['customer_id', 'product_unit_variant_id', 'week_start']].assign(qty_this_week=0.0)
], ignore_index=True)

full_df = full_df.sort_values(['customer_id', 'product_unit_variant_id', 'week_start'])

# B. Add "Grandmaster Features" (Seasonality & Trends)
# Fix: Handle potential NaT values before converting to int
full_df['month'] = full_df['week_start'].dt.month.fillna(0).astype(int)
full_df['week_of_year'] = full_df['week_start'].dt.isocalendar().week.fillna(0).astype(int)

# C. Global Product Trend (Shifted to prevent leakage)
# "How much is this product selling globally across all customers?"
prod_trend = full_df.groupby(['product_unit_variant_id', 'week_start'])['qty_this_week'].mean().reset_index().rename(columns={'qty_this_week': 'global_qty'})
prod_trend = prod_trend.sort_values(['product_unit_variant_id', 'week_start'])
prod_trend['global_trend_lag1'] = prod_trend.groupby('product_unit_variant_id')['global_qty'].shift(1)
full_df = full_df.merge(prod_trend[['product_unit_variant_id', 'week_start', 'global_trend_lag1']],
                        on=['product_unit_variant_id', 'week_start'], how='left')

# D. Lag Features
print("Generating lags...")
grp = full_df.groupby(['customer_id', 'product_unit_variant_id'])['qty_this_week']
full_df['lag1'] = grp.shift(1)
full_df['lag2'] = grp.shift(2)
full_df['roll_mean_4'] = grp.rolling(4).mean().reset_index(level=[0,1], drop=True)

# --- 4. Target Generation (CRITICAL FIX) ---
# We generate targets ON THE FULL DATASET so the shifts work correctly.
print("Generating Targets on Full Data...")
full_df['target_qty_1w'] = grp.shift(-1) # Next week's qty
full_df['target_qty_2w'] = grp.shift(-2) # Week after next qty

# --- 5. Merge Metadata ---
# Now we bring in Customer and SKU details
full_df = full_df.merge(customer, on='customer_id', how='left')
full_df = full_df.merge(sku, on='product_unit_variant_id', how='left')

# Encode Categoricals
cat_cols = ['customer_category', 'customer_status', 'grade_name', 'unit_name']
le = LabelEncoder()
for col in cat_cols:
    full_df[col] = full_df[col].astype(str).fillna('UNKNOWN')
    full_df[col] = le.fit_transform(full_df[col])

# Fill Numerical NaNs
num_cols = ['lag1', 'lag2', 'roll_mean_4', 'global_trend_lag1']
full_df[num_cols] = full_df[num_cols].fillna(0)

# --- 6. Create Stacked Training Data ---
print("Stacking Data...")

# Identify Train rows vs Test rows
# Train rows are those that were in the original Train CSV
train_mask = full_df['week_start'].isin(train['week_start'].unique())
train_df_source = full_df[train_mask].copy()
test_df_source = full_df[~train_mask].copy()

# Stack 1: Horizon 1
h1 = train_df_source.copy()
h1['target_qty'] = h1['target_qty_1w']
h1['target_buy'] = (h1['target_qty'] > 0).astype(int)
h1['horizon'] = 1

# Stack 2: Horizon 2
h2 = train_df_source.copy()
h2['target_qty'] = h2['target_qty_2w']
h2['target_buy'] = (h2['target_qty'] > 0).astype(int)
h2['horizon'] = 2

# Combine
train_stacked = pd.concat([h1, h2], ignore_index=True)

# Drop rows where target is NaN (End of history)
train_stacked = train_stacked.dropna(subset=['target_qty'])

# Define Features
features = ['lag1', 'lag2', 'roll_mean_4', 'global_trend_lag1',
            'month', 'week_of_year', 'horizon',
            'customer_category', 'customer_status', 'grade_name', 'unit_name']

# --- 7. Modeling ---
print("Training Unified Models...")

# Classifier (AUC)
clf = lgb.LGBMClassifier(n_estimators=2000, learning_rate=0.03, num_leaves=31, random_state=42)
clf.fit(train_stacked[features], train_stacked['target_buy'])

# Regressor (Tweedie for MAE)
reg = lgb.LGBMRegressor(objective='tweedie', tweedie_variance_power=1.5,
                        n_estimators=2000, learning_rate=0.03, num_leaves=31, random_state=42)
reg.fit(train_stacked[features], train_stacked['target_qty'])

# --- 8. Prediction ---
print("Generating Predictions...")

# Prepare Test sets
test_h1 = test_df_source.copy()
test_h1['horizon'] = 1

test_h2 = test_df_source.copy()
test_h2['horizon'] = 2

# Predict
submission = test[['ID']].copy()
submission['Target_purchase_next_1w'] = clf.predict_proba(test_h1[features])[:, 1]
submission['Target_qty_next_1w'] = reg.predict(test_h1[features])
submission['Target_purchase_next_2w'] = clf.predict_proba(test_h2[features])[:, 1]
submission['Target_qty_next_2w'] = reg.predict(test_h2[features])

# --- 9. Consistency Check ---
# If probability of buy is very low, force quantity to 0?
# (Optional, but helps MAE). Let's be conservative.
# submission.loc[submission['Target_purchase_next_1w'] < 0.1, 'Target_qty_next_1w'] = 0
# submission.loc[submission['Target_purchase_next_2w'] < 0.1, 'Target_qty_next_2w'] = 0

submission.to_csv('submission_horizon_fixed.csv', index=False)
print("Done! Saved as submission_horizon_fixed.csv")

Loading data...
Building Full History...
Generating lags...
Generating Targets on Full Data...
Stacking Data...
Training Unified Models...


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
from sklearn.preprocessing import LabelEncoder

# --- 1. Load Data ---
print("Loading data...")
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
customer = pd.read_csv('customer_data.csv')
sku = pd.read_csv('sku_data.csv')

# --- 2. Downcast to Save Memory ---
def downcast(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    for col in df.select_dtypes(include=['int64']).columns:
        if df[col].max() < 2**32:
            df[col] = df[col].astype('int32')
    return df

train = downcast(train)
test = downcast(test)
customer = downcast(customer)
sku = downcast(sku)

# Convert Dates
train['week_start'] = pd.to_datetime(train['week_start'])
test['week_start'] = pd.to_datetime(test['week_start'])
customer['customer_created_at'] = pd.to_datetime(customer['customer_created_at'])

# --- 3. Unified Feature Engineering ---
print("Building Full History...")

# A. Combine Train and Test FIRST
# We need the full timeline to calculate trends and lags correctly
full_df = pd.concat([
    train[['customer_id', 'product_unit_variant_id', 'week_start', 'qty_this_week']],
    test[['customer_id', 'product_unit_variant_id', 'week_start']].assign(qty_this_week=0.0)
], ignore_index=True)

full_df = full_df.sort_values(['customer_id', 'product_unit_variant_id', 'week_start'])

# --- GRANDMASTER FEATURES ---

# 1. Seasonality (Time)
full_df['month'] = full_df['week_start'].dt.month.fillna(0).astype(int)
full_df['week_of_year'] = full_df['week_start'].dt.isocalendar().week.fillna(0).astype(int)

# 2. Global Product Trend (The "Viral" Factor)
# Calculates: "How much is this specific product selling across ALL customers right now?"
# We shift by 1 week so we don't peek into the future.
prod_trend = full_df.groupby(['product_unit_variant_id', 'week_start'])['qty_this_week'].mean().reset_index().rename(columns={'qty_this_week': 'global_qty'})
prod_trend = prod_trend.sort_values(['product_unit_variant_id', 'week_start'])
prod_trend['global_trend_lag1'] = prod_trend.groupby('product_unit_variant_id')['global_qty'].shift(1)
prod_trend['global_trend_roll4'] = prod_trend.groupby('product_unit_variant_id')['global_qty'].transform(lambda x: x.shift(1).rolling(4).mean())

full_df = full_df.merge(prod_trend[['product_unit_variant_id', 'week_start', 'global_trend_lag1', 'global_trend_roll4']],
                        on=['product_unit_variant_id', 'week_start'], how='left')

# 3. Lag Features (Individual History)
print("Generating lags...")
grp = full_df.groupby(['customer_id', 'product_unit_variant_id'])['qty_this_week']
full_df['lag1'] = grp.shift(1)
full_df['lag2'] = grp.shift(2)
full_df['lag3'] = grp.shift(3)
full_df['roll_mean_4'] = grp.rolling(4).mean().reset_index(level=[0,1], drop=True)
full_df['roll_max_4'] = grp.rolling(4).max().reset_index(level=[0,1], drop=True)

# --- 4. Target Generation (THE FIX) ---
# Generate targets on the full timeline BEFORE splitting
print("Generating Targets...")
full_df['target_qty_1w'] = grp.shift(-1) # Next week's qty
full_df['target_qty_2w'] = grp.shift(-2) # Week after next qty

# --- 5. Merge Metadata & Encode ---
full_df = full_df.merge(customer, on='customer_id', how='left')
full_df = full_df.merge(sku, on='product_unit_variant_id', how='left')

# Customer Tenure (How long have they been with us?)
full_df['customer_tenure_days'] = (full_df['week_start'] - full_df['customer_created_at']).dt.days

# Encode Categoricals
cat_cols = ['customer_category', 'customer_status', 'grade_name', 'unit_name']
le = LabelEncoder()
for col in cat_cols:
    full_df[col] = full_df[col].astype(str).fillna('UNKNOWN')
    full_df[col] = le.fit_transform(full_df[col])

# Fill NaNs in features with 0
num_cols = ['lag1', 'lag2', 'lag3', 'roll_mean_4', 'roll_max_4', 'global_trend_lag1', 'global_trend_roll4', 'customer_tenure_days']
full_df[num_cols] = full_df[num_cols].fillna(0)

# --- 6. Create Stacked Training Data ---
print("Stacking Data...")

# Filter back to Train and Test sets
train_mask = full_df['week_start'].isin(train['week_start'].unique())
train_df_source = full_df[train_mask].copy()
test_df_source = full_df[~train_mask].copy()

# Stack 1: Horizon 1 (Predicting Next Week)
h1 = train_df_source.copy()
h1['target_qty'] = h1['target_qty_1w']
h1['target_buy'] = (h1['target_qty'] > 0).astype(int)
h1['horizon'] = 1

# Stack 2: Horizon 2 (Predicting 2 Weeks out)
h2 = train_df_source.copy()
h2['target_qty'] = h2['target_qty_2w']
h2['target_buy'] = (h2['target_qty'] > 0).astype(int)
h2['horizon'] = 2

# Combine Stacks
train_stacked = pd.concat([h1, h2], ignore_index=True)
train_stacked = train_stacked.dropna(subset=['target_qty']) # Valid targets only

# Define Features
features = [
    'lag1', 'lag2', 'lag3', 'roll_mean_4', 'roll_max_4', # Individual behavior
    'global_trend_lag1', 'global_trend_roll4',           # Market behavior
    'month', 'week_of_year', 'horizon',                  # Time context
    'customer_tenure_days',                              # Loyalty
    'customer_category', 'customer_status', 'grade_name', 'unit_name' # Metadata
]

# --- 7. Modeling (High Precision) ---
print("Training Unified Models...")

# Classifier (Optimized for AUC)
# Lower learning rate (0.02) + More trees (2500) = Better Pattern Recognition
clf = lgb.LGBMClassifier(
    n_estimators=2500,
    learning_rate=0.02,
    num_leaves=40,
    random_state=42,
    subsample=0.8,
    colsample_bytree=0.8
)
clf.fit(train_stacked[features], train_stacked['target_buy'])
print("Classifier Trained.")

# Regressor (Optimized for Tweedie/MAE)
reg = lgb.LGBMRegressor(
    objective='tweedie',
    tweedie_variance_power=1.5,
    n_estimators=2500,
    learning_rate=0.02,
    num_leaves=40,
    random_state=42,
    subsample=0.8,
    colsample_bytree=0.8
)
reg.fit(train_stacked[features], train_stacked['target_qty'])
print("Regressor Trained.")

# --- 8. Prediction ---
print("Generating Predictions...")

# Prepare Test sets for both horizons
test_h1 = test_df_source.copy()
test_h1['horizon'] = 1

test_h2 = test_df_source.copy()
test_h2['horizon'] = 2

submission = test[['ID']].copy()

# Horizon 1 Predictions
submission['Target_purchase_next_1w'] = clf.predict_proba(test_h1[features])[:, 1]
submission['Target_qty_next_1w'] = reg.predict(test_h1[features])

# Horizon 2 Predictions
submission['Target_purchase_next_2w'] = clf.predict_proba(test_h2[features])[:, 1]
submission['Target_qty_next_2w'] = reg.predict(test_h2[features])

# Final Cleanup (No negative quantities)
submission['Target_qty_next_1w'] = submission['Target_qty_next_1w'].clip(lower=0)
submission['Target_qty_next_2w'] = submission['Target_qty_next_2w'].clip(lower=0)

submission.to_csv('submission_horizon_promax.csv', index=False)
print("Done! Saved as submission_horizon_promax.csv")

Loading data...
Building Full History...
Generating lags...
Generating Targets...
Stacking Data...
Training Unified Models...
Classifier Trained.
Regressor Trained.
Generating Predictions...


In [ ]:
# --- 7. Validation & Modeling (With Scores!) ---
print("--- Starting Validation ---")

# A. Create a Time-Based Split for Validation
# We use the last 4 weeks of the training data to check our score
weeks = sorted(train_stacked['week_start'].unique())
val_start_week = weeks[-4]

train_subset = train_stacked[train_stacked['week_start'] < val_start_week]
val_subset = train_stacked[train_stacked['week_start'] >= val_start_week]

# B. Train & Score Classifier (AUC)
print("Validating Classifier...")
clf_val = lgb.LGBMClassifier(
    n_estimators=2000,
    learning_rate=0.02,
    num_leaves=40,
    random_state=42,
    subsample=0.8,
    colsample_bytree=0.8
)
clf_val.fit(
    train_subset[features],
    train_subset['target_buy'],
    eval_set=[(val_subset[features], val_subset['target_buy'])],
    eval_metric='auc',
    callbacks=[lgb.early_stopping(stopping_rounds=100)]
)

# Calculate Validation AUC
val_preds_prob = clf_val.predict_proba(val_subset[features])[:, 1]
from sklearn.metrics import roc_auc_score, mean_absolute_error
val_auc = roc_auc_score(val_subset['target_buy'], val_preds_prob)
print(f"✅ LOCAL VALIDATION AUC: {val_auc:.5f}")

# C. Train & Score Regressor (MAE)
print("Validating Regressor...")
reg_val = lgb.LGBMRegressor(
    objective='tweedie',
    tweedie_variance_power=1.5,
    n_estimators=2000,
    learning_rate=0.02,
    num_leaves=40,
    random_state=42,
    subsample=0.8,
    colsample_bytree=0.8
)
reg_val.fit(
    train_subset[features],
    train_subset['target_qty'],
    eval_set=[(val_subset[features], val_subset['target_qty'])],
    eval_metric='mae',
    callbacks=[lgb.early_stopping(stopping_rounds=100)]
)

# Calculate Validation MAE
val_preds_qty = reg_val.predict(val_subset[features])
val_mae = mean_absolute_error(val_subset['target_qty'], val_preds_qty)
print(f"✅ LOCAL VALIDATION MAE: {val_mae:.5f}")

# --- 8. Final Retraining & Prediction ---
print("\n--- Retraining on FULL Dataset for Submission ---")
# Now that we know the score, we train on EVERYTHING (Train + Validation) to get max performance
clf_full = lgb.LGBMClassifier(
    n_estimators=2500,
    learning_rate=0.02,
    num_leaves=40,
    random_state=42,
    subsample=0.8,
    colsample_bytree=0.8
)
clf_full.fit(train_stacked[features], train_stacked['target_buy'])

reg_full = lgb.LGBMRegressor(
    objective='tweedie',
    tweedie_variance_power=1.5,
    n_estimators=2500,
    learning_rate=0.02,
    num_leaves=40,
    random_state=42,
    subsample=0.8,
    colsample_bytree=0.8
)
reg_full.fit(train_stacked[features], train_stacked['target_qty'])

print("Generating Final Predictions...")

# Prepare Test sets for both horizons
test_h1 = test_df_source.copy()
test_h1['horizon'] = 1

test_h2 = test_df_source.copy()
test_h2['horizon'] = 2

submission = test[['ID']].copy()

# Horizon 1 Predictions
submission['Target_purchase_next_1w'] = clf_full.predict_proba(test_h1[features])[:, 1]
submission['Target_qty_next_1w'] = reg_full.predict(test_h1[features])

# Horizon 2 Predictions
submission['Target_purchase_next_2w'] = clf_full.predict_proba(test_h2[features])[:, 1]
submission['Target_qty_next_2w'] = reg_full.predict(test_h2[features])

# Final Cleanup (No negative quantities)
submission['Target_qty_next_1w'] = submission['Target_qty_next_1w'].clip(lower=0)
submission['Target_qty_next_2w'] = submission['Target_qty_next_2w'].clip(lower=0)

submission.to_csv('submission_horizon_promax_scored.csv', index=False)
print("Done! Saved as submission_horizon_promax_scored.csv")

In [35]:
# --- ADD THIS TO YOUR FEATURE ENGINEERING STEP ---

# 1. Extract Date Features
# These help the model learn seasonality (e.g., mango season)
train['month'] = train['week_start'].dt.month
train['week_of_year'] = train['week_start'].dt.isocalendar().week.astype(int)
train['quarter'] = train['week_start'].dt.quarter

test['month'] = test['week_start'].dt.month
test['week_of_year'] = test['week_start'].dt.isocalendar().week.astype(int)
test['quarter'] = test['week_start'].dt.quarter

# 2. Calculate Customer Tenure (Days on Platform)
# Older customers behave differently than new ones
train['tenure_days'] = (train['week_start'] - train['customer_created_at']).dt.days
test['tenure_days'] = (test['week_start'] - test['customer_created_at']).dt.days

# 3. Add these new columns to your 'features' list
# (Update the list in the Modeling Step)
# features = ['month', 'week_of_year', 'tenure_days', 'customer_category', ...] + feature_cols

ValueError: cannot convert NA to integer